In [277]:
import pandas as pd

In [278]:
df = pd.read_csv('../data/synthetic_clinical_logs.txt', sep='|', names=['Patient name', 'Temp (celsius)', 'BP', 'Notes'])

In [279]:
df = pd.DataFrame(df)

In [280]:
df['Patient name'] = df['Patient name'].str.replace('Patient: ', '')

df['Temp (celsius)'] = df['Temp (celsius)'].str.replace('Temp: ', '').str.replace('°C', '').str.strip()
df['Temp (celsius)'] = pd.to_numeric(df['Temp (celsius)'], errors='coerce')

df['BP'] = df['BP'].str.replace('BP: ', '').str.replace('not taken','None')

df['Notes'] = df['Notes'].str.replace('Notes: ', '').str.replace('"','')

In [281]:
df.head(50)

,Patient name,Temp (celsius),BP,Notes
0,Erica Kennedy,39.0,127/64,Mild cough
1,Kiara Fisher,NaN,127/62,headache and nausea
2,Douglas Benton,NaN,135/85,complained of fatigue
3,William Maldonado,NaN,None,complained of fatigue
4,Nicole Velazquez,36.8,142/71,chest pain
5,Tyler Hinton,38.1,109/92,
6,Terri Cook,39.2,144/74,no complaints
7,Jason Wise,NaN,126/71,shortness of breath
8,Jorge Harmon,NaN,None,chest pain
9,Nicholas Walker,39.7,119/78,
